In [1]:
from pyspark.sql.types import StructType
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
import math
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
my_conf = SparkConf()
my_conf.set("spark.app.name", "w12_Spark_1")
my_conf.set("spark.master", "local[*]")
my_conf.set("spark.driver.extraClassPath", "C:/Users/Vinoth/Downloads/sqljdbc_12.4.2.0_enu/sqljdbc_12.4/enu/jars/mssql-jdbc-12.4.2.jre8.jar")

spark = SparkSession.builder.config(conf = my_conf).getOrCreate()
spark


In [3]:
database_url = "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true"

database_name = "Learn_Sql"

jdbc_properties = {

"user": "Vinoth",

"password": "1234",

"driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"

}

In [4]:
# df_min_max = (spark.read
# .format("jdbc")
# .option("url", "jdbc:sqlserver://DESKTOP-FJ7E01E\SQLEXPRESS:1433;encrypt=true;trustServerCertificate=true;databaseName=Learn_Sql;")
# .option("query","select min(cost) as min_row, max(cost) as max_row from products")
# .option("user", "Vinoth")
# .option("password", "1234")
# .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver")
# .load().collect())

# min_value, max_value = df_min_max[0][0], df_min_max[0][1]

In [5]:
# df = (spark.read
#       .option("numPartitions", 3)
#       .option("PartitionColumn", "cost")
#       .option("lowerBound", int(min_value))
#       .option("upperBound", int(max_value))
#       .jdbc(url=f"{database_url};databaseName={database_name};", table="products", properties=jdbc_properties))

In [6]:
input_df = (spark.read
      .jdbc(url=f"{database_url};databaseName={database_name};", table="Ameriprise_LLC", properties=jdbc_properties))


In [14]:
input_df.show()

+------+--------+---------+---------+
|teamID|memberID|Criteria1|Criteria2|
+------+--------+---------+---------+
|    T1| T1_mbr1|        Y|        Y|
|    T1| T1_mbr2|        Y|        Y|
|    T1| T1_mbr3|        Y|        Y|
|    T1| T1_mbr4|        Y|        Y|
|    T1| T1_mbr5|        Y|        N|
|    T2| T2_mbr1|        Y|        Y|
|    T2| T2_mbr2|        Y|        N|
|    T2| T2_mbr3|        N|        Y|
|    T2| T2_mbr4|        N|        N|
|    T2| T2_mbr5|        N|        N|
|    T3| T3_mbr1|        Y|        Y|
|    T3| T3_mbr2|        Y|        Y|
|    T3| T3_mbr3|        N|        Y|
|    T3| T3_mbr4|        N|        Y|
|    T3| T3_mbr5|        Y|        N|
+------+--------+---------+---------+



In [13]:
df_agg = input_df.\
   withColumn("flag", expr("case when Criteria1 == 'Y' and Criteria2 == 'Y' then 'Y' else 'N' end"))\
.where("flag = 'Y'")\
.groupBy("teamID").agg(expr("count(*) as cnt")).where("cnt >=2")

In [26]:
input_df.alias("t1").join(df_agg.alias("t2"), col("t1.teamID") == col("t2.teamID"), 'left')\
.withColumn("output", expr("case when Criteria1 == 'Y' and Criteria2 == 'Y' and cnt is not null then 'Y' else 'N' end"))\
.orderBy("t1.teamID").select("t1.teamID", "t1.memberID", "t1.Criteria1", "t1.Criteria2", "output").show()

+------+--------+---------+---------+------+
|teamID|memberID|Criteria1|Criteria2|output|
+------+--------+---------+---------+------+
|    T1| T1_mbr1|        Y|        Y|     Y|
|    T1| T1_mbr2|        Y|        Y|     Y|
|    T1| T1_mbr3|        Y|        Y|     Y|
|    T1| T1_mbr4|        Y|        Y|     Y|
|    T1| T1_mbr5|        Y|        N|     N|
|    T2| T2_mbr1|        Y|        Y|     N|
|    T2| T2_mbr2|        Y|        N|     N|
|    T2| T2_mbr3|        N|        Y|     N|
|    T2| T2_mbr4|        N|        N|     N|
|    T2| T2_mbr5|        N|        N|     N|
|    T3| T3_mbr1|        Y|        Y|     Y|
|    T3| T3_mbr2|        Y|        Y|     Y|
|    T3| T3_mbr3|        N|        Y|     N|
|    T3| T3_mbr4|        N|        Y|     N|
|    T3| T3_mbr5|        Y|        N|     N|
+------+--------+---------+---------+------+

